In [ ]:
# import tensorflow as tf
# # Enable mixed precision for faster training and reduced memory usage
# from tensorflow.keras.mixed_precision import set_global_policy
# set_global_policy('mixed_float16')

# # GPU memory growth to prevent fragmentation
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, DepthwiseConv2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

2025-01-28 20:34:33.793366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-28 20:34:34.835713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-12.6/lib64
2025-01-28 20:34:34.835851: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7

In [2]:
# ✅ Step 1: Constants
IMG_SIZE = (48, 48)  # Match original dataset resolution
NUM_CLASSES = 8      # Update based on your dataset
BATCH_SIZE = 32       # Reduced batch size to fit GPU memory
EPOCHS = 50
DATASET_DIR = "/home/natalyagrokh/img_datasets/combo_ferckja_dataset_2"

In [3]:
# ✅ Step 2: Define Early Stopping & Learning Rate Reduction
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

In [4]:
# ✅ Step 3: Augment Underrepresented Classes (Before Dataset Loading)
def augment_and_save(class_name, multiplier):
    """Augments images in the given class folder to increase dataset size."""
    class_path = os.path.join(DATASET_DIR, class_name)
    images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.jpg', '.png'))]
    current_count = len(images)
    target_count = current_count * multiplier
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    for i in range(target_count - current_count):
        img_path = images[i % current_count]
        img = tf.keras.preprocessing.image.load_img(img_path)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape for ImageDataGenerator
        
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=class_path, save_prefix="aug", save_format="jpg"):
            break  # Only generate one image per iteration

# ✅ Apply augmentation to rare classes BEFORE loading dataset
augment_and_save("contempt", 30)  # 30x more images for Contempt
augment_and_save("disgust", 5)    # 5x more images for Disgust

In [5]:
# ✅ Step 4: Define Optimized Model (Reduce Overfitting)
# Step 4: Define Simplified Model
model = Sequential([
    # First Convolution Block
    Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Second Convolution Block
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    # Fully Connected Layers
    Flatten(),
    Dense(128, activation="relu"),  # Reduced Dense layer size
    Dropout(0.5),
    Dense(NUM_CLASSES, activation="softmax")
])

model.summary()

2025-01-28 20:35:10.506182: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-28 20:35:10.541616: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-28 20:35:10.544997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-28 20:35:10.548425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 64)       2

In [6]:
# ✅ Step 5: Compile Model with Optimized Learning Rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,  # Higher learning rate for better updates
    decay_steps=10000,          # Larger decay steps
    decay_rate=0.9              # Slower decay rate
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [7]:
# ✅ Step 6: Configure Model Checkpoint
model_checkpoint = ModelCheckpoint(
    filepath="best_emotion_model_weights.h5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,  # Save only weights to prevent optimizer errors
    verbose=1
)

In [8]:
# ✅ Step 7: Load Training and Validation Datasets (After Augmentation)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels="inferred",
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="grayscale",
    validation_split=0.2,
    subset="training",
    seed=42
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels="inferred",
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode="grayscale",
    validation_split=0.2,
    subset="validation",
    seed=42
)

# ✅ Apply Data Augmentation Pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

# Apply Augmentation During Training
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x) / 255.0, y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Normalize Validation Data
val_dataset = val_dataset.map(
    lambda x, y: (x / 255.0, y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Prefetch for Performance
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

Found 51057 files belonging to 8 classes.
Using 40846 files for training.
Found 51057 files belonging to 8 classes.
Using 10211 files for validation.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
# ✅ Step 8: Compute Class Weights and Train Model
labels = np.concatenate([y.numpy().argmax(axis=1) for _, y in train_dataset])
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)
class_weights = {i: w for i, w in enumerate(class_weights)}

# Train Model with Class Weights
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1
)

Epoch 1/50


2025-01-28 20:35:55.084237: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2025-01-28 20:35:55.482768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2025-01-28 20:35:55.860024: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f9bd44c2560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-28 20:35:55.860064: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-01-28 20:35:55.866389: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-28 20:35:56.006725: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled clust

1277/1277 [==============================] - ETA: 0s - loss: 2.1040 - accuracy: 0.1106     
Epoch 1: val_accuracy improved from -inf to 0.10538, saving model to best_emotion_model_weights.h5
1277/1277 [==============================] - 42s 30ms/step - loss: 2.1040 - accuracy: 0.1106 - val_loss: 2.0798 - val_accuracy: 0.1054 - lr: 9.8665e-04
Epoch 2/50
1273/1277 [============================>.] - ETA: 0s - loss: 2.0800 - accuracy: 0.1068  
Epoch 2: val_accuracy improved from 0.10538 to 0.10910, saving model to best_emotion_model_weights.h5
1277/1277 [==============================] - 38s 30ms/step - loss: 2.0801 - accuracy: 0.1067 - val_loss: 2.0784 - val_accuracy: 0.1091 - lr: 9.7346e-04
Epoch 3/50
1275/1277 [============================>.] - ETA: 0s - loss: 2.0796 - accuracy: 0.1047  
Epoch 3: val_accuracy did not improve from 0.10910
1277/1277 [==============================] - 36s 28ms/step - loss: 2.0796 - accuracy: 0.1047 - val_loss: 2.0803 - val_accuracy: 0.1031 - lr: 9.6045e-04


In [ ]:
# Step 9: Save the Final Model
weights_path = "best_emotion_model_weights.h5"

# ✅ Check if weights file exists before loading
if os.path.exists(weights_path):
    print("✅ Loading best model weights...")
    model.load_weights(weights_path)
else:
    print("⚠️ Warning: No weights file found. Skipping weight loading.")

# ✅ Save the full model
model.save("final_emotion_model.keras")  # TensorFlow 2.11+ prefers `.keras`
print("✅ Training complete and final model saved.")